### Controlar el acceso de red a S3 desde su VPC utilizando puntos finales de VPC

#### Problema
Los recursos dentro de su VPC deben ser capaces de acceder sólo a un cubo S3 específico. Además, este tráfico de S3 no debe atravesar Internet por razones de seguridad y para mantener bajos los costes de ancho de banda.

#### Solución
Creará un punto final de la VPC de gateway para S3, lo asociará con una tabla de rutas y personalizará su documento de políticas.

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0213.png" width="500">

In [1]:
import boto3
import json
import time

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)

In [2]:
# create VPC
vpc = ec2.create_vpc(CidrBlock='174.17.0.0/16')
vpc.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC"}])
vpc.wait_until_available()

In [3]:
# Crear routable
routetable_1 = vpc.create_route_table()
routetable_2 = vpc.create_route_table()
routetable_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbookVPC-RT-1"}])
routetable_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbookVPC-RT-2"}])

[ec2.Tag(resource_id='rtb-06dec9332ddedc283', key='Name', value='AWSCookbookVPC-RT-2')]

In [4]:
# Cree dos subredes:
subnet_1 = ec2.create_subnet(
    CidrBlock='174.17.0.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1a'
)

subnet_2 = ec2.create_subnet(
    CidrBlock='174.17.1.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1b'
)

subnet_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-SN-Private-1"}])
subnet_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-SN-Private-2"}])

[ec2.Tag(resource_id='subnet-06a0a8ba06f52c432', key='Name', value='AWSCookbook-SN-Private-2')]

In [5]:
# Asociar las tablas de rutas con las subred:
routetable_1.associate_with_subnet(SubnetId = subnet_1.id)
routetable_2.associate_with_subnet(SubnetId = subnet_2.id)

ec2.RouteTableAssociation(id='rtbassoc-0a9b4b4ac2346dcf7')

In [6]:
import random
# generate random number
random_number_1 = random.randint(100, 10000)
random_number_2 = random.randint(100, 1000)

In [7]:
# Crear un bucket de S3
s3 = boto3.resource('s3')
s3_name = 'awscookbookbucket-' + str(random_number_1) + '-' + str(random_number_2)

try:
    bucket = s3.create_bucket(
        Bucket=s3_name,
    )
except Exception as e:
    print(e)
else:
    print('Bucket created: {} '.format(bucket.name))

Bucket created: awscookbookbucket-6523-197 


In [8]:
print("Crear una instancia EC2 con permisos SSM")
print("En la terminal ejecute:")
print("    python create_ec2_ssm.py --vpc {}".format(vpc.id))

Crear una instancia EC2 con permisos SSM
En la terminal ejecute:
    python create_ec2_ssm.py --vpc vpc-0736f29a836ec7e55


Cree un gateway-endpoint en su VPC y asocie el endpoint con las tablas de rutas aisladas:


In [9]:
gateway_endpoint = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.{}.s3'.format(region_aws),
    VpcEndpointType='Gateway',
    RouteTableIds=[
        routetable_1.id,
        routetable_2.id
    ]
)


Cree un archivo de política de punto final de plantilla llamado policy.json con el siguiente contenido. Esto se utiliza para limitar el acceso sólo al bucket de S3 que creó en los pasos de preparación:

In [10]:
policy_endpoint = {
  "Statement": [
    {
      "Sid": "RestrictToOneBucket",
      "Principal": "*",
      "Action": [
        "s3:GetObject",
        "s3:PutObject"
      ],
      "Effect": "Allow",
      "Resource": ["arn:aws:s3:::" + s3_name + "/",
                   "arn:aws:s3:::" + s3_name + "/*"]
    }
  ]
}

Modificar el documento de políticas del endpoint. Las políticas del endpoint limitan o restringen los recursos a los que se puede acceder a través del endpoint de la VPC:

In [11]:
ec2_client.modify_vpc_endpoint(
    VpcEndpointId=gateway_endpoint['VpcEndpoint']['VpcEndpointId'],
    PolicyDocument=json.dumps(policy_endpoint)
)

{'Return': True,
 'ResponseMetadata': {'RequestId': 'e9349ff2-d74b-4c2a-a2d3-d2974de79303',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e9349ff2-d74b-4c2a-a2d3-d2974de79303',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'accept-encoding',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'date': 'Mon, 10 Oct 2022 21:54:21 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

#### Comprobaciones de validación

Imprime el nombre de tu S3 Bucket para que puedas referirte a él cuando estés conectado a tu Instancia EC2:

In [33]:
# Subir un archivo de prueba al bucket de S3
bucket.upload_file('ip-ranges.json', 'test_file')

In [12]:
print(s3_name)

awscookbookbucket-6523-197


Conéctese a la instancia EC2 mediante el gestor de sesiones SSM 

In [37]:
print("En la terminal ejecute:")
print("    python create_ec2_ssm.py --last")

En la terminal ejecute:
    python create_ec2_ssm.py --last


A la instancia SSM otórguele permisos para acceder al bucket s3 

En la instancia ejecute:

In [ ]:
export AWS_DEFAULT_REGION=$(curl \
--silent http://169.254.169.254/latest/dynamic/instance-identity/document \
| awk -F'"' ' /region/ {print $4}')

In [ ]:
BUCKET_NAME="<BUCKET_NAME>"

El comando a continuación debería ejecutarse correctamente si la instancia tiene los roles correctos para acceder a S3:

In [ ]:
aws s3 cp s3://${BUCKET_NAME}/test_file /home/ssm-user/

El siguiente comando está intentando listar un bucket S3 público. Sin embargo, debido a la política de punto final que hemos configurado, se espera que esto falle:

In [ ]:
aws s3 ls s3://osm-pds/

#### Discusión
El uso de una política de punto final es una implementación de seguridad útil para restringir el acceso a los cubos de S3. Esto se aplica no sólo a los cubos de S3 propiedad de su cuenta, sino también a todos los cubos de S3 a nivel global en AWS.

**TIP**
Recientemente, AWS ha anunciado la compatibilidad con los puntos finales de la interfaz de S3. Sin embargo, vale la pena señalar que si bien estos son grandes para algunos casos de uso (por ejemplo, cuando se desea controlar el tráfico con los grupos de seguridad), no son ideales para este problema debido a los costos asociados con los puntos finales de la interfaz.

Según la Guía del usuario de la VPC, los puntos finales de la VPC de gateway son gratuitos y se utilizan dentro de las tablas de rutas de su VPC para mantener el tráfico destinado a los servicios de AWS dentro de la red troncal de AWS sin atravesar la red. Esto le permite crear VPCs que no necesitan gateways de Internet para aplicaciones que no los requieren pero que necesitan acceso a otros servicios de AWS como S3 y DynamoDB. Todo el tráfico destinado a estos servicios será dirigido por la tabla de rutas al punto final de la VPC en lugar de la ruta pública de Internet, ya que la entrada de la tabla de rutas del punto final de la VPC es más específica que la ruta predeterminada 0.0.0.0/0.

Las políticas del punto final de la VPC S3 aprovechan los documentos de política JSON que pueden ser tan detallados como sus necesidades lo requieran. Puede utilizar condicionales, direcciones IP de origen, ID de puntos finales de VPC, nombres de cubos de S3, etc. Para obtener más información sobre los elementos de política disponibles, consulte el documento de soporte.


